In [130]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
from scipy.stats import hypergeom
import scipy.stats as ss

In [100]:
tsv = 'targets_human.tsv'
xlsx = 'EMT.xlsx'
header = ['Transcription Factor', 'Target', 'Effect', 'Nums']
df = pd.read_csv(tsv, sep='\t', names=header, index_col=0)
df_EMT = pd.read_excel(xlsx, index_col=0)

In [101]:
df_EMT.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Type
Gene,,,,,,,
CD44,3395.584314,-3.604432,0.153022,-23.555054,1.114243e-122,2.450556e-118,protein_coding
SGCB,168.140875,-1.491506,0.139296,-10.707437,9.392604e-27,7.945059e-24,protein_coding
DAB2,343.670369,-1.582603,0.175692,-9.007814,2.102036e-19,7.578702e-17,protein_coding
SERPINE1,257.551780,-1.665060,0.194868,-8.544546,1.290426e-17,3.835182e-15,protein_coding
COL5A1,143.003939,-2.134700,0.276503,-7.720355,1.160058e-14,2.108525e-12,protein_coding


In [102]:
df.head()

,Target,Effect,Nums
Transcription Factor,,,
AATF,BAX,Repression,22909821
AATF,CDKN1A,Unknown,17157788
AATF,KLK3,Unknown,23146908
AATF,MYC,Activation,20549547
AATF,TP53,Unknown,17157788


In [103]:
target = df.Target.unique() # M
gene = df_EMT.index.unique() # n
M = len(target)
n = len(gene)
TF = df.index.unique()

In [140]:
tf = []
intersec = []
TRRUST_genes = []
EMT_genes = []
TRRUST_tf_genes = []
p_value = []
l = []

In [141]:
for name in TF:
    sel = df[df.index == name]
    unique_targets = sel.Target.unique() # N
    N = len(unique_targets)
    inter = list(set(gene) & set(unique_targets)) # k
    l.append(inter)
    k = len(inter)
    rv = hypergeom(M, n, N)
    pv = rv.sf(k)
    
    tf.append(name)
    intersec.append(len(inter))
    TRRUST_genes.append(M)
    EMT_genes.append(n)
    TRRUST_tf_genes.append(N)
    p_value.append(pv)

In [144]:
final_df = pd.DataFrame({'Transcription Factor': tf, 'Intersected genes': intersec, 'TRRUST unique': TRRUST_genes, 
                         'EMT unique': EMT_genes, 'TRRUST unique targets': TRRUST_tf_genes, 'p_value': p_value, 'Genes': l})
final_df = final_df.set_index('Transcription Factor')

In [145]:
final_df['padj'] = np.minimum(final_df['p_value'] * len(final_df) / ss.rankdata(final_df['p_value']), 1)
final_df = final_df.sort_values('padj')
final_df

,Intersected genes,TRRUST unique,EMT unique,TRRUST unique targets,p_value,Genes,padj
Transcription Factor,,,,,,,
SSX2,1,2492,107,1,0.000000,[MMP2],0.000000
HOXD1,1,2492,107,1,0.000000,[ITGB1],0.000000
TEAD4,1,2492,107,1,0.000000,[VEGFA],0.000000
TCF7,1,2492,107,1,0.000000,[SNAI2],0.000000
ZFP36L1,1,2492,107,1,0.000000,[VEGFA],0.000000
...,...,...,...,...,...,...,...
YY1,1,2492,107,91,0.910403,[COL1A2],0.915007
GATA1,0,2492,107,57,0.920398,[],0.923301
FOS,0,2492,107,57,0.920398,[],0.923301


In [ ]:
column_names = [Intersected genes	TRRUST unique	EMT unique	TRRUST unique targets	p_value	Genes	padj]

df = df.reindex(columns=column_names)

In [146]:
final_df.to_csv('TRRUST_vs_EMT.csv', index=True)